In [2]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

In [3]:
# State that holds conversation history
class MyMessagesState(MessagesState):
    pass

In [4]:
# Node: the AI generates a response and may call a tool
def tool_calling_llm(state: MyMessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [6]:
# Tools
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product."""
    return a * b

def translate_to_french(text: str) -> str:
    """Return a mock French translation of the input text."""
    return f"French translation of '{text}'"

def get_weather(city: str) -> str:
    """Return a mock weather string for the given city."""
    return f"The weather in {city} is sunny."

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5")

# Bind all tools
llm_with_tools = llm.bind_tools([multiply, translate_to_french, get_weather])

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [7]:
# Build the graph
builder = StateGraph(MyMessagesState)

# Add nodes
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, translate_to_french, get_weather]))

# Connect nodes
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", END)

In [8]:
# Compile
graph = builder.compile()